In [12]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [13]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [14]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [15]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [16]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [17]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [18]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.301s

OK


In [19]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [20]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [21]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [24]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.209s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [26]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
passphrase = b'programmingbitcoinbitshalacohort@josedorrego.com very very long secret that only I know'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret)
# change address should be the address generated from Chapter 4
change_address = 'muUcyfA8J5xQncsbhBMfE65UjiwunMTHnp'

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx = bytes.fromhex('21d07bcb3762a429beec93a2eef44b41dd9ba0506b7a67f0aa8a5b04322730f8')
prev_index = 1

# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
target_amount = 0.00000600
change_amount = 0.00000100


# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100000000)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(change_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
print(tx_obj.sign_input(0, priv))
print(tx_obj.serialize().hex())

# broadcast at http://testnet.blockchain.info/pushtx
# testnet.blockchain.info/pushtx is not working right now, so went for this other one:
# https://live.blockcypher.com/btc-testnet/pushtx/
# https://live.blockcypher.com/btc-testnet/tx/338f98906e585f52ccfe991b57ff7cbce5c0451e212cdb16a50ec75f2e597aa8/

True
0100000001f8302732045b8aaaf0677a6b50a09bdd414bf4eea293ecbe29a46237cb7bd021010000006a47304402200494ad97dcf27a72bbeffe366d11e031f8caf532fa40253f60027c0820b5a20e022033f0f8c6da22720591cc160e6876574779a59e0fe72867b146ac6332567f4f95012102c30f89957515bc922ff2efb5b8a62c9af33a18f57ed5c851d4d6b58b36b71dcdffffffff0258020000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac64000000000000001976a9149920cdcc1aae1536e6abb5e9d6a1f8e0d877416388ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [28]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
passphrase = b'programmingbitcoinbitshalacohort@josedorrego.com very very long secret that only I know'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx_1 = bytes.fromhex('d4d60f3345a06f0cb6437cb2a9b4bf9607ce910b233bc69e787c47d32de8cf4e')
prev_index_1 = 0

# create the first transaction input with the default ScriptSig and
# sequence
tx_ins = []
tx_ins.append(TxIn(prev_tx_1, prev_index_1))

# get the prev_tx and prev_index from the transaction in Exercise 4
prev_tx_2 = bytes.fromhex('338f98906e585f52ccfe991b57ff7cbce5c0451e212cdb16a50ec75f2e597aa8')
prev_index_2 = 1

# create the second transaction input with the default ScriptSig and
# sequence
tx_ins.append(TxIn(prev_tx_2, prev_index_2))

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
target_amount = 0.00001

# create a transaction output for the amount and address
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100000000)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign the first input using the private key
print(tx_obj.sign_input(0, priv))
# sign the second input using the private key 
print(tx_obj.sign_input(1, priv))
# print the transaction's serialization in hex
print(tx_obj.serialize().hex())

# broadcast at http://testnet.blockchain.info/pushtx
# testnet.blockchain.info/pushtx is not working right now, so went for this other one:
# https://live.blockcypher.com/btc-testnet/pushtx/
# https://live.blockcypher.com/btc-testnet/tx/5320441c8af29c8f582d6d7199ee046369e5674a58da3ff563abc302f189df6e/

True
True
01000000024ecfe82dd3477c789ec63b230b91ce0796bfb4a9b27c43b60c6fa045330fd6d4000000006b483045022100c056e7f6944c86b4466167cc29ea0f3bb97c056afc39154e80ba8d53b254710d02207240e998c5136b23265bbe890f489f251c90de54f59f019c53843c3bb49cb749012102c30f89957515bc922ff2efb5b8a62c9af33a18f57ed5c851d4d6b58b36b71dcdffffffffa87a592e5fc70ea516db2c211e45c0e5bc7cff571b99fecc525f586e90988f33010000006a473044022040468ea8526f51621d10e80e1402cf232615bdac85674eafa0a426962074874f022003b08fc6444a2248f978489447c3a5d7cd470b727c6b378d499948c45981d15c012102c30f89957515bc922ff2efb5b8a62c9af33a18f57ed5c851d4d6b58b36b71dcdffffffff01e8030000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
